Задача: Выделить основные темы публикаций научного журнала по аннотациям, предварительно их почистив

In [1]:
#pip install keras

In [2]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
english_stopwords = stopwords.words("english")
import string
spec_chars = string.punctuation + '«»—…’‘”“©'
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
from tqdm import tqdm
import numpy as np
tf_idf_vectorizer = TfidfVectorizer(tokenizer=lambda doc: doc, lowercase=False)
cv_vectorizer = CountVectorizer(tokenizer=lambda doc: doc, lowercase=False)
import pandas as pd
pd.set_option('display.max_columns', None)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sofiarebrey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sofiarebrey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:

df = pd.read_excel('gender.xlsx', names = ['author', 'uni', 'title', 'concept', 'methodology',
              'output', 'abstract', 'keywords', 'year', 'volume'])

In [4]:
df=df['abstract']


In [5]:
df=df.dropna()

In [6]:
#Почистим аннотации от стоп слов, уберем пунктуацию, лемматезируем тексты
descriptions = df.to_list()

stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer() 



In [7]:
# Метод для очистки текстов, лематизации и нормализации
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)  
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    normalized = re.sub(r'(?:[\W\d_])+', ' ', normalized)
    return normalized

clean_descriptions = [clean(doc).split() for doc in descriptions]

In [8]:
#Что мы тут делаем?
tf_idf_vectorizer = TfidfVectorizer(tokenizer=lambda doc: doc, lowercase=False)
cv_vectorizer = CountVectorizer(tokenizer=lambda doc: doc, lowercase=False)
tf_idf_arr = tf_idf_vectorizer.fit_transform(clean_descriptions)
cv_arr = cv_vectorizer.fit_transform(clean_descriptions)

In [9]:
#Составляем словари в векторизованном виде

vocab_tf_idf = tf_idf_vectorizer.get_feature_names()
vocab_cv = cv_vectorizer.get_feature_names()



In [10]:
#Генерируем 100 возможных тем, которые будут описывать тексты

TOPIC_NUM = 100
lda_model = LatentDirichletAllocation(n_components = TOPIC_NUM, max_iter = 20, random_state = 42)
X_topics = lda_model.fit_transform(tf_idf_arr)
topic_words = lda_model.components_

In [11]:

#Каждая тема будет состоять из 5 самых популярных слов

n_top_words = 5

topics = {}

for i, topic_dist in enumerate(topic_words):
    sorted_topic_dist = np.argsort(topic_dist)
    topic_words = np.array(vocab_tf_idf)[sorted_topic_dist]
    topic_words = topic_words[:-n_top_words:-1]
    topics[i+1] = topic_words

In [12]:
#Собираем темы для каждого текста и генерируем дамми признаки на основе ключевых слов 
#каждой темы

all_topics = []

doc_topic = lda_model.transform(tf_idf_arr)  
for n in range(doc_topic.shape[0]):
    topic_doc = doc_topic[n].argmax()
    _t = [t for t in list(topics[topic_doc+1]) if len(t) > 2]
    all_topics.append(",".join(_t))

df_topics = pd.DataFrame(np.array(all_topics))
df_topics = df_topics[0].str.get_dummies(',')
df_topics

,abortion,access,acms,adjusted,adoption,african,agency,agricultural,agriculture,agriculturenutrition,american,appropriate,asset,associated,asymmetry,attitude,australia,background,ban,bank,bargaining,behavior,birth,book,breastfeeding,burden,business,cambodia,capital,caregiver,caring,cater,cbla,childcare,cleaner,cohort,completion,computes,concept,continuous,contraception,could,council,country,credit,csg,cua,culture,defeminization,demand,democracy,depending,difference,different,discrimination,distribution,divorce,domestic,east,ecological,economics,eldercare,elderly,elite,empathy,empirical,employed,employee,endogeneity,enrollment,equality,essay,expansionary,export,extension,external,extradomestic,facto,farm,farmwork,fathers,feminist,feminization,fertility,field,firmfinanced,formal,gender,genderwage,generation,german,germany,girl,gpi,grabbing,growth,headscarf,headscarved,held,hiv,holistic,hour,household,housework,human,identity,illegal,india,inequality,injustice,institution,integrity,intrahousehold,ipv,jure,kenya,labor,land,leave,liberty,life,limited,list,literature,loan,local,loss,lunch,macroeconomic,malawi,management,manager,manufacturing,marriage,mayor,mental,method,microfinance,mostly,neoliberalism,nepal,nicaragua,nonelite,nonfarm,norm,notion,nrpp,nutrition,offered,offfarm,opt,paid,paper,parent,parental,parttime,party,pathway,penalty,pension,performed,poverty,power,prevention,price,private,problem,production,productive,proportion,prosperity,randomista,ratio,recipient,recovery,relation,relationship,remittance,reproductive,reservation,review,rise,risk,sam,satisfaction,save,saving,scenario,scholar,school,schooling,secondary,sectoral,segregation,servants,sex,share,shift,shifted,side,skill,smoker,smoking,social,son,sourcecountry,south,spanish,state,statistic,statistical,still,study,suggest,switzerland,task,teenage,tertiary,time,title,total,trade,tradeoff,trafficking,training,turkey,twelvemonth,understand,unilateral,unpaid,use,variation,vaw,view,village,violence,voice,wage,waring,wealth,woman,worker,workforce,year,zealand
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [13]:
all_topics

['business,randomista,elite,nonelite',
 'land,side,cambodia,grabbing',
 'segregation,leave,sectoral,parental',
 'nutrition,agriculture,agricultural,pathway',
 'nepal,facto,jure',
 'party,list,south,csg',
 'headscarf,ban,tertiary,headscarved',
 'reproductive,saving,empirical,recovery',
 'wage,abortion,reservation,offered',
 'employed,turkey,contraception,method',
 'african,waring,american,statistic',
 'fertility,sourcecountry,generation,norm',
 'productive,defeminization,trade,extradomestic',
 'mental,offfarm,burden,parent',
 'land,side,cambodia,grabbing',
 'son,land,schooling,access',
 'bargaining,power,cua,injustice',
 'identity,nonfarm,agency,india',
 'labor,household,country,paper',
 'party,list,south,csg',
 'workforce,share,manager,private',
 'pension,management,title,discrimination',
 'woman,gender,study',
 'trafficking,girl,ratio,illegal',
 'agriculturenutrition,time,use,tradeoff',
 'housework,eldercare,domestic,satisfaction',
 'feminist,equality,economics,growth',
 'violence,dom

Получается какая-то чушь... 

In [14]:











#Собираем темы для каждого текста и генерируем дамми признаки на основе ключевых слов каждой темы

all_topics = []

doc_topic = lda_model.transform(tf_idf_arr)  
for n in range(doc_topic.shape[0]):
    topic_doc = doc_topic[n].argmax()
    _t = [t for t in list(topics[topic_doc+1]) if len(t) > 2]
    all_topics.append(",".join(_t))

df_topics = pd.DataFrame(np.array(all_topics))
df_topics = df_topics[0].str.get_dummies(',')
df_topics

,abortion,access,acms,adjusted,adoption,african,agency,agricultural,agriculture,agriculturenutrition,american,appropriate,asset,associated,asymmetry,attitude,australia,background,ban,bank,bargaining,behavior,birth,book,breastfeeding,burden,business,cambodia,capital,caregiver,caring,cater,cbla,childcare,cleaner,cohort,completion,computes,concept,continuous,contraception,could,council,country,credit,csg,cua,culture,defeminization,demand,democracy,depending,difference,different,discrimination,distribution,divorce,domestic,east,ecological,economics,eldercare,elderly,elite,empathy,empirical,employed,employee,endogeneity,enrollment,equality,essay,expansionary,export,extension,external,extradomestic,facto,farm,farmwork,fathers,feminist,feminization,fertility,field,firmfinanced,formal,gender,genderwage,generation,german,germany,girl,gpi,grabbing,growth,headscarf,headscarved,held,hiv,holistic,hour,household,housework,human,identity,illegal,india,inequality,injustice,institution,integrity,intrahousehold,ipv,jure,kenya,labor,land,leave,liberty,life,limited,list,literature,loan,local,loss,lunch,macroeconomic,malawi,management,manager,manufacturing,marriage,mayor,mental,method,microfinance,mostly,neoliberalism,nepal,nicaragua,nonelite,nonfarm,norm,notion,nrpp,nutrition,offered,offfarm,opt,paid,paper,parent,parental,parttime,party,pathway,penalty,pension,performed,poverty,power,prevention,price,private,problem,production,productive,proportion,prosperity,randomista,ratio,recipient,recovery,relation,relationship,remittance,reproductive,reservation,review,rise,risk,sam,satisfaction,save,saving,scenario,scholar,school,schooling,secondary,sectoral,segregation,servants,sex,share,shift,shifted,side,skill,smoker,smoking,social,son,sourcecountry,south,spanish,state,statistic,statistical,still,study,suggest,switzerland,task,teenage,tertiary,time,title,total,trade,tradeoff,trafficking,training,turkey,twelvemonth,understand,unilateral,unpaid,use,variation,vaw,view,village,violence,voice,wage,waring,wealth,woman,worker,workforce,year,zealand
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0